In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dt=pd.read_csv("spam1.csv")
dt.head(10)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [ ]:
dt['spam']=dt['Category'].map({'spam':1, 'ham':0}).astype(int)
dt.head(5)
#dt.drop(['label'],axis=1)

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
print("COLUMNS IN THE GIVEN DATA:")
for col in dt.columns:
    print(col)

COLUMNS IN THE GIVEN DATA:
Category
Message
spam


In [ ]:
t=len(dt['Category'])
print("No of rows in review column:",t)
t=len(dt['Message'])
print("No of rows in liked column",t)

No of rows in review column: 5572
No of rows in liked column 5572


In [ ]:
#Tokenization
dt['Message'][1]

'Ok lar... Joking wif u oni...'

In [ ]:
def tokenizer(text):
    return text.split()


In [ ]:
dt['Message']=dt['Message'].apply(tokenizer)

In [ ]:
dt['Message'][1]

['Ok', 'lar...', 'Joking', 'wif', 'u', 'oni...']

In [ ]:
from nltk.stem.snowball import SnowballStemmer
porter = SnowballStemmer("english", ignore_stopwords=False)

In [ ]:
def stem_it(text):
    return [porter.stem(word) for word in text]

In [ ]:
dt['Message']=dt['Message'].apply(stem_it)

In [ ]:
dt['Message'][1]

['ok', 'lar...', 'joke', 'wif', 'u', 'oni...']

In [ ]:
#Lemmitization
dt['Message'][152]#before

['yup',
 'i',
 'thk',
 'cine',
 'is',
 'better',
 'cos',
 'no',
 'need',
 '2',
 'go',
 'down',
 '2',
 'plaza',
 'mah.']

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmit_it(text):
    return [lemmatizer.lemmatize(word, pos='a') for word in text]

In [ ]:
dt['Message']=dt['Message'].apply(lemmit_it)

In [ ]:
dt['Message'][152]

['yup',
 'i',
 'thk',
 'cine',
 'is',
 'good',
 'cos',
 'no',
 'need',
 '2',
 'go',
 'down',
 '2',
 'plaza',
 'mah.']

In [ ]:
#Stopword Removal
dt['Message'][217]

['tired.', 'i', "haven't", 'slept', 'well', 'the', 'past', 'few', 'nights.']

In [ ]:
from nltk.corpus import stopwords
stop_words=stopwords.words('english')

In [ ]:
def stop_it(text):
    review=[word for word in text if not word in stop_words]
    return review

In [ ]:
dt['Message']=dt['Message'].apply(stop_it)

In [ ]:
dt['Message'][217]

['tired.', 'slept', 'well', 'past', 'nights.']

In [ ]:
dt.head(10)

,Category,Message,spam,label
0,ham,"[go, jurong, point,, crazy.., avail, onli, bug...",0,0
1,ham,"[ok, lar..., joke, wif, u, oni...]",0,0
2,spam,"[free, entri, 2, wkli, comp, win, fa, cup, fin...",1,1
3,ham,"[u, dun, say, earli, hor..., u, c, alreadi, sa...",0,0
4,ham,"[nah, think, goe, usf,, live, around, though]",0,0
5,spam,"[freemsg, hey, darl, 3, week, word, back!, i'd...",1,1
6,ham,"[even, brother, like, speak, me., treat, like,...",0,0
7,ham,"[per, request, mell, mell, (oru, minnaminungin...",0,0
8,spam,"[winner!!, valu, network, custom, select, rece...",1,1
9,spam,"[mobil, 11, month, more?, u, r, entitl, updat,...",1,1


In [ ]:
dt['Message']=dt['Message'].apply(' '.join)

In [ ]:
dt.head()

,Category,Message,spam,label
0,ham,"go jurong point, crazy.. avail onli bugi n gre...",0,0
1,ham,ok lar... joke wif u oni...,0,0
2,spam,free entri 2 wkli comp win fa cup final tkts 2...,1,1
3,ham,u dun say earli hor... u c alreadi say...,0,0
4,ham,"nah think goe usf, live around though",0,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
y=dt.spam.values
x=tfidf.fit_transform(dt['Message'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1,test_size=0.2,shuffle=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)
from sklearn.metrics import accuracy_score
acc_log=accuracy_score(y_pred, y_test)*100
print("accuracy:",acc_log)

accuracy: 96.05381165919282


In [ ]:
from sklearn.svm import LinearSVC
linear_svc=LinearSVC(random_state=0)
linear_svc.fit(x_train,y_train)
y_pred=linear_svc.predict(x_test)
acc_linear_svc=accuracy_score(y_pred,y_test)*100
print("accuracy:",acc_linear_svc)

accuracy: 97.66816143497758


In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier(random_state=0)
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)
from sklearn.metrics import accuracy_score
acc_dec=accuracy_score(y_pred,y_test)*100
print("accuracy:",acc_dec)

accuracy: 96.7713004484305


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
nv=KNeighborsClassifier(n_neighbors=3)
nv.fit(x_train,y_train)
y_pred=nv.predict(x_test)
from sklearn.metrics import accuracy_score
acc_knn=accuracy_score(y_pred,y_test)*100
print("accuracy:",acc_knn)

accuracy: 92.01793721973094


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=10)
rf.fit(x_train,y_train)
y_pred=rf.predict(x_test)
from sklearn.metrics import accuracy_score
acc_rand=accuracy_score(y_pred,y_test)*100
print("accuracy:",acc_rand)

accuracy: 96.7713004484305
